# AgentOps Langchain Agent Implementation

Using AgentOps monitoring with Langchain is simple. We've created a LangchainCallbackHandler that will do all of the heavy lifting!

First we'll import the typical Langchain packages:

In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.agents import tool

The only difference with using AgentOps is that we'll also import this special Callback Handler

In [3]:
from agentops.partners.langchain_callback_handler import (
    LangchainCallbackHandler as AgentOpsLangchainCallbackHandler,
)

Next, we'll grab our two API keys. You can use dotenv like below or however else you like to load environment variables

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

This is where AgentOps comes into play. Before creating our LLM instance via Langchain, first we'll create an instance of the AO LangchainCallbackHandler. After the handler is initialized, a session will be recorded automatically.

Pass in your API key, and optionally any tags to describe this session for easier lookup in the AO dashboard.

In [5]:
AGENTOPS_API_KEY = os.environ.get("AGENTOPS_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

agentops_handler = AgentOpsLangchainCallbackHandler(
    api_key=AGENTOPS_API_KEY, tags=["Langchain Example"]
)

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, callbacks=[agentops_handler], model="gpt-3.5-turbo"
)

You can also retrieve the `session_id` of the newly created session.

In [6]:
print("Agent Ops session ID: " + str(agentops_handler.current_session_ids))

Agent Ops session ID: []


Agents generally use tools. Let's define a simple tool here. Tool usage is also recorded.

In [7]:
@tool
def find_movie(genre) -> str:
    """Find available movies"""
    if genre == "drama":
        return "Dune 2"
    else:
        return "Pineapple Express"


tools = [find_movie]

For each tool, you need to also add the callback handler

In [8]:
for t in tools:
    t.callbacks = [agentops_handler]

Finally, let's use our agent! Pass in the callback handler to the agent, and all the actions will be recorded in the AO Dashboard

In [9]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    callbacks=[
        agentops_handler
    ],  # You must pass in a callback handler to record your agent
    handle_parsing_errors=True,
)

/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [16]:
agent.invoke("What comedies are playing?", callbacks=[agentops_handler])

Error in LangchainCallbackHandler.on_chain_start callback: ValueError()
Error in LangchainCallbackHandler.on_llm_start callback: ValueError()
Error in LangchainCallbackHandler.on_llm_error callback: KeyError('adbe88bd-c2bc-4738-8e79-a0f93899b8df')
Error in LangchainCallbackHandler.on_chain_error callback: KeyError('bfbb3d30-6120-4b87-9367-885ec5f63ea2')




> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************JL4Y. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

## Check your session
Finally, check your run on [AgentOps](https://app.agentops.ai)
![image.png](attachment:3d9393fa-3d6a-4193-b6c9-43413dc19d15.png)

# Async Agents

Several langchain agents require async callback handlers. AgentOps also supports this.

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.agents import tool

In [ ]:
from agentops.partners.langchain_callback_handler import (
    AsyncLangchainCallbackHandler as AgentOpsAsyncLangchainCallbackHandler,
)

In [ ]:
from dotenv import load_dotenv

load_dotenv()

AGENTOPS_API_KEY = os.environ.get("AGENTOPS_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [ ]:
agentops_handler = AgentOpsAsyncLangchainCallbackHandler(
    api_key=AGENTOPS_API_KEY, tags=["Async Example"]
)

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, callbacks=[agentops_handler], model="gpt-3.5-turbo"
)

print("Agent Ops session ID: " + str(await agentops_handler.session_id))

In [ ]:
@tool
def find_movie(genre) -> str:
    """Find available movies"""
    if genre == "drama":
        return "Dune 2"
    else:
        return "Pineapple Express"


tools = [find_movie]

for t in tools:
    t.callbacks = [agentops_handler]

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    callbacks=[agentops_handler],
)

await agent.arun("What comedies are playing?")

## Check your session
Finally, check your run on [AgentOps](https://app.agentops.ai)

![image.png](attachment:69f2121a-d437-4c09-bbbe-c76c9243ee19.png)